# Graph Construction

This notebook construct the academic graph into `torch.geometric.data` object's format given the citation information file.

#### Settings
The citation information file should have the format:
todo


#### Prepration

In [3]:
import os
%load_ext autoreload
%autoreload 2

#### Set up dataset directory:

In [4]:
cwd = os.getcwd()
dataset_dir = os.path.join(cwd, "dataset", "CUHKSZ_AcademicGraph")
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

#### To load our dataset:
0. Library dependency: pandas, torch, torch_geometric
1. Put `CUHKSZ_AcademicGraph.py` into your module utility folder.
2. Import and create a CUHKSZ_AcademicGraph opject.

In [136]:
from utils.CUHKSZ_AcademicGraph import CUHKSZ_AcademicGraph

dataset = CUHKSZ_AcademicGraph(root=dataset_dir, with_label=True)

Processing...


d:\github\CUHKSZ_AcademicGraph\dataset\CUHKSZ_AcademicGraph\raw\CUHKSZ_AcademicGraph_Rawdata.zip
d:\github\CUHKSZ_AcademicGraph\dataset\CUHKSZ_AcademicGraph\raw\CUHKSZ_AcademicGraph-rawdata_released


FileNotFoundError: [Errno 2] No such file or directory: 'd:\\github\\CUHKSZ_AcademicGraph\\dataset\\CUHKSZ_AcademicGraph\\raw\\CUHKSZ_AcademicGraph-rawdata_released\\CUHKSZ_AcademicGraph_Rawdata.zip'

In [102]:
dataset

CUHKSZ_AcademicGraph()

In [103]:
dataset[0].y

array([1, 1, 5, ..., 0, 0, 0], dtype=int64)

In [113]:
import pandas as pd
import ast
import numpy as np
unzip_dir = r"./dataset/CUHKSZ_AcademicGraph/raw/CUHKSZ_AcademicGraph-rawdata_released"

fact_df = pd.read_csv(os.path.join(unzip_dir, "fact_df.csv"))
mask = fact_df["0"].values == 0

In [134]:
raw_paper_info = pd.read_csv(os.path.join(unzip_dir, "Raw_Paper_Info.csv"))
raw_paper_info = raw_paper_info[mask]
raw_paper_info = raw_paper_info.sort_values(by=["paperId"])
raw_paper_info = raw_paper_info.reset_index()
raw_paper_info = raw_paper_info.drop(["index"], axis=1)
raw_paper_info.to_csv("Raw_Paper_Info.csv", index=None)
# raw_paper_info

raw_embedding = pd.read_csv(os.path.join(unzip_dir, "Raw_Paper_Embedding.csv"))
raw_embedding = raw_embedding.sort_values(by=["paperId"])
raw_embedding = raw_embedding.reset_index()
raw_embedding = raw_embedding.drop(["index"], axis=1)
raw_embedding.to_csv("Raw_Paper_Embedding.csv", index=None)

In [98]:
import pandas as pd
import ast
import numpy as np

unzip_dir = r"./dataset/CUHKSZ_AcademicGraph/raw/CUHKSZ_AcademicGraph-rawdata_released"
raw_paper_info = pd.read_csv(os.path.join(unzip_dir, "Raw_Paper_Info.csv"))
raw_paper_info.columns = raw_paper_info.columns.str.strip()
valid = raw_paper_info["fieldsOfStudy"].notna()
raw_paper_info = raw_paper_info[valid]

tags = []
for i, tag_list_str in enumerate(raw_paper_info["fieldsOfStudy"].values):
    tag_list = ast.literal_eval(tag_list_str)
    tags.append(tag_list[0])
raw_paper_info["fieldsOfStudy"] = tags
top8_labels = raw_paper_info["fieldsOfStudy"].value_counts()[:8]
in_top8 = raw_paper_info["fieldsOfStudy"].map(lambda x:x in top8_labels)
raw_paper_info = raw_paper_info[in_top8]
raw_paper_info = raw_paper_info.reset_index()

top8_labels_map = {}
for i, field in enumerate(top8_labels.index):
    top8_labels_map[field] = i
raw_paper_info["fieldsOfStudy"] = raw_paper_info["fieldsOfStudy"].map(lambda x: top8_labels_map[x])
y = raw_paper_info["fieldsOfStudy"].values

In [99]:
valid

0        True
1        True
2        True
3        True
4        True
        ...  
7719     True
7720    False
7721     True
7722    False
7723    False
Name: fieldsOfStudy, Length: 7724, dtype: bool

In [100]:
raw_embedding = pd.read_csv(os.path.join(unzip_dir, "Raw_Paper_Embedding.csv"))
raw_embedding

,paperId,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,3fa4638cae1b36951ba0c23ce6723ee0eb66aa7a,-6.800751,-4.985931,-2.179117,0.942490,0.101148,-2.876867,0.007298,-2.180235,0.002225,...,0.448072,-1.391343,-0.913795,3.461416,-1.048731,-0.498397,5.157135,-3.854411,0.526945,-3.338611
1,c259a61877ec300ccb6fa3b1878c197bbb0e9402,0.273761,-2.210204,0.671116,-2.200044,-2.007438,-0.763784,0.740047,-6.864379,1.872152,...,2.107108,-1.439257,0.116833,4.161168,1.143128,-1.074347,1.397650,-1.536129,-2.075511,-1.613855
2,25167ffa3486ec5dd31f09f1bd9f5efce50e5568,-1.188344,-0.691393,0.943876,0.378633,-4.464087,-0.700293,5.401897,-1.808418,2.915785,...,0.177523,-3.882308,-1.456316,0.805596,-1.693231,-1.912326,3.865959,-2.855784,4.201446,-3.179596
3,ec8a3f0b6b130c6a940b7245c0123bd5944fca80,-5.893249,-3.951370,0.101285,4.436233,-2.815700,-0.259500,3.911988,-3.816236,1.120150,...,3.995578,-2.363170,3.062009,1.935523,-0.921990,-1.366561,1.405898,1.320966,1.811619,-4.002594
4,f5bd8bc45a9151aef0b2e48982d3f35544fe390d,-2.978854,0.585969,-2.905960,1.706044,-2.677280,-2.419511,2.610264,-5.641686,-0.328582,...,5.068091,-1.247999,8.907757,3.109290,2.609141,1.086094,2.088407,-1.585195,-1.571840,-1.448218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7624,6d4bbb2a5e01963b1cec185a194695fa9ae4420e,-5.529283,-2.306182,-2.334910,3.630635,3.402728,1.674935,3.910689,-3.404171,-3.199624,...,-1.822102,3.115554,0.699631,1.034564,1.487842,2.134938,2.790453,-3.236255,0.770500,-0.577812
7625,6fb0fb0ac224d3d6346d2340f8f948cba075ce21,-2.745112,-0.487711,-1.874027,3.019404,-3.472379,-2.147450,1.501764,-5.849020,3.960171,...,-1.756435,-0.979145,2.240037,2.654546,-2.521661,-4.567371,2.250682,2.394383,-0.347625,-2.665774
7626,d3a48a146e124c76fe76e5234071722b9681793a,-4.226266,-1.778911,-1.059561,1.052593,-2.794808,1.698553,6.716641,0.053719,0.814782,...,-2.913860,1.052951,-0.724687,-0.064086,-3.761267,-0.168765,1.506511,-2.363891,-1.757677,0.725629
7627,b7d50f04592a6aa361214356d4efd225a1214058,3.339037,-3.229419,4.085373,-1.488225,0.206440,1.081175,-0.532840,-2.773635,3.061046,...,1.544669,-1.847220,3.927692,-0.966215,-2.027585,1.726042,0.876773,-2.327452,-5.650959,-0.308709
